# Migration check

## Supporting documentation
- [Introduction to the Python Driver for Oracle Database](https://python-oracledb.readthedocs.io/en/latest/user_guide/introduction.html#introduction-to-the-python-driver-for-oracle-database)
- [Python python-oracledb Driver](https://oracle.github.io/python-oracledb/)
- [Python: Read Data from Oracle Database](https://kontext.tech/article/1019/python-read-data-from-oracle-database)

## Connecting to the database via the ORACLE "thin" client

In [1]:
    import oracledb
    import getpass
    
    username = "CUSTDOC"
    userpwd = getpass.getpass("Enter password: ")

    host="localhost"
    port="1521"
    service_name="XEPDB1"
    dsn = f'{username}/{userpwd}@{host}:{port}/{service_name}'
   

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


Enter password:  ········


## Clearing the table ```CUSTDOC.CUST$DOCS$URLS```


In [ ]:
import oracledb
import os

try:
    connection = oracledb.connect(dsn)
    cursor = connection.cursor()
    sqld="""DELETE FROM CUSTDOC.CUST$DOCS$URLS"""
    cursor.execute(sqld) 
    connection.commit();
except oracledb.Error as e:
    error_obj, = e.args
    print(f"Oracle Error: {error_obj.code} - {error_obj.message}")
except Exception as error:
    print(f"Unexpected Error: {type(error).__name__}")
    print(f"ErrorDetails: {error}")
    print(sys.exc_info())    
finally:
    if cursor:
        cursor.close()
        print("\nThe cursor is closed.")
    if connection:
        connection.close()
        print("The connection to the database has been closed.")

## Clearing a table by condition (by FLID)

In [ ]:
import oracledb
import os

try:
    connection = oracledb.connect(dsn)
    cursor = connection.cursor()
    sqld="""DELETE FROM CUSTDOC.CUST$DOCS$URLS WHERE IDDOC=:A_IDDOC"""
    cursor.execute(sqld, { "A_IDDOC":  2563 }) 
    connection.commit();
except oracledb.Error as e:
    error_obj, = e.args
    print(f"Oracle Oracle: {error_obj.code} - {error_obj.message}")
except Exception as error:
    print(f"Unexpected Error: {type(error).__name__}")
    print(f"Error details: {error}")
    print(sys.exc_info())    
finally:
    if cursor:
        cursor.close()
        print("\nThe cursor is closed.")
    if connection:
        connection.close()
        print("The connection to the database has been closed.")

#### Viewing test data

In [ ]:
import pandas as pd
from IPython.display import display

sql_query = """
    SELECT A.IDFL, A.IDDOC, A.FILE_NAME, A.CONTENT_TYPE, A.FILE_SIZE, A.FILE_URL, A.IDT, A.IUSRNM
    FROM CUSTDOC.CUST$DOCS$URLS  A
 """

connection = None
cursor = None
pd.set_option('display.max_columns', None)  # Відображати всі стовпці
pd.set_option('display.width', 3000)        # Збільшити ширину консолі для відображення

try:
    connection = oracledb.connect(dsn=dsn)
    cursor = connection.cursor()
    cursor.execute(sql_query)
    column_names = [col[0] for col in cursor.description]
    rows = cursor.fetchall()
    if rows:
        df = pd.DataFrame(rows, columns=column_names)
        display(df)
        print(f"Fetched rows={len(rows)}")
    else:
        print("The query did not return any rows.")
except oracledb.Error as e:
    error_obj, = e.args
    print(f"Помилка Oracle: {error_obj.code} - {error_obj.message}")
finally:
    if cursor:
        cursor.close()
        print("\nThe cursor is closed.")
    if connection:
        connection.close()
        print("The connection to the database has been closed.")

# Search for not yet migrated records in the database

In [ ]:
import pandas as pd
from IPython.display import display

sql_query = """
    SELECT A.CUSTID, A.IDDOC, A.ISACRUAL FROM CUSTDOC.CUST$DOCS  A 
        WHERE NOT EXISTS( SELECT 1 FROM CUSTDOC.CUST$DOCS$URLS B WHERE  B.IDDOC=A.IDDOC) 
 """
connection = None
cursor = None
pd.set_option('display.max_columns', None)  # Відображати всі стовпці
pd.set_option('display.width', 3000)        # Збільшити ширину консолі для відображення

try:
    connection = oracledb.connect(dsn=dsn)
    cursor = connection.cursor()
    cursor.execute(sql_query)
    column_names = [col[0] for col in cursor.description]
    rows = cursor.fetchall()
    if rows:
        df = pd.DataFrame(rows, columns=column_names)
        display(df)
        print(f"Fetched rows={len(rows)}")
    else:
        print("The query did not return any rows.")
except oracledb.Error as e:
    error_obj, = e.args
    print(f"Помилка Oracle: {error_obj.code} - {error_obj.message}")
finally:
    if cursor:
        cursor.close()
        print("\nThe cursor is closed.")
    if connection:
        connection.close()
        print("The connection to the database has been closed.")

# Aggregated migration display

In [ ]:
import pandas as pd
from IPython.display import display
sql_query = """
    SELECT 'NUMBER OF DOCUMENTS' as METRICS, COUNT(*) AS RECORD_COUNT 
    FROM CUSTDOC.CUST$DOCS  A
    UNION ALL
    SELECT 'JUST SENT NOT CONFIRMED' as METRICS, COUNT(*) AS RECCNT  
        FROM CUSTDOC.CUST$DOCS  A
        WHERE NOT EXISTS( SELECT 1 FROM CUSTDOC.CUST$DOCS$URLS B WHERE  B.IDDOC=A.IDDOC) 
        AND A.ISACRUAL='N'
    UNION ALL
    SELECT 'MIGRATED AND CONFIRMED' as METRICS, COUNT(*) AS RECCNT  
        FROM CUSTDOC.CUST$DOCS  A
        WHERE  EXISTS( SELECT 1 FROM CUSTDOC.CUST$DOCS$URLS B WHERE  B.IDDOC=A.IDDOC) 
        AND A.ISACRUAL='N'        
    UNION ALL
    SELECT 'NOT MIGRATED' as METRICS, COUNT(*) AS RECCNT  
        FROM CUSTDOC.CUST$DOCS  A
        WHERE NOT EXISTS( SELECT 1 FROM CUSTDOC.CUST$DOCS$URLS B WHERE  B.IDDOC=A.IDDOC) 
        AND A.ISACRUAL='Y'
 """
connection = None
cursor = None
pd.set_option('display.max_columns', None)  # Відображати всі стовпці
pd.set_option('display.width', 3000)        # Збільшити ширину консолі для відображення
try:
    connection = oracledb.connect(dsn=dsn)
    cursor = connection.cursor()
    cursor.execute(sql_query)
    column_names = [col[0] for col in cursor.description]
    rows = cursor.fetchall()
    if rows:
        df = pd.DataFrame(rows, columns=column_names)
        display(df)
        print(f"Fetched rows={len(rows)}")
    else:
        print("The query did not return any rows.")
except oracledb.Error as e:
    error_obj, = e.args
    print(f"Oracle Error: {error_obj.code} - {error_obj.message}")
finally:
    if cursor:
        cursor.close()
        print("\nThe cursor is closed.")
    if connection:
        connection.close()
        print("The connection to the database has been closed.")